# Census API Metric Codes

| Race | Code |
|------|------|
| Total|B03002_001E|
| Black|B03002_004E|
| Asian|B03002_006E|
| Native Hawaiian Pacific Islander|B03002_007E|
| Other|B03002_008E|
| Hispanic or Latino|B03002_012E|
| 2 or More Races|B03002_010E|

| Citizenship | Code |
|------|------|
| Total| B05001_001E
| Not a u.s. Citizen|B05001_006E|

| Income | Code |
|------|------|
| Total income population|B05010_001E|
| Under poverty line|B05010_002E|

| Education | Code |
|------|------|
| Less than HS graduate |B07009_002E|
| High school graduate |B07009_003E|
| Some college or associate's degree |B07009_004E|
| Grad or professional degree |B07009_006E|

| Under 5 | Code |
|------|------|
| Total |B01001_001E|
| Male under 5 |B01001_003E|
| Female under 5 |B01001_027E|

| Housing | Code |
|------|------|
| Total |B07013_001E|
| Renters |B07013_003E|


In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import geopandas as gpd
import requests

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import (ColumnDataSource,HoverTool, LogColorMapper)
from bokeh.palettes import Viridis6 as palette
from bokeh.tile_providers import STAMEN_TERRAIN
output_notebook()

#If you need to install anything
#import sys
#!conda install --yes --prefix {sys.prefix} numpy

Loading BokehJS ...

In [2]:
#Read in local files
#fcc = pd.read_csv("/Users/robertdeng/Google Drive/Data Science/W209/Project/fbd_us_without_satellite_dec2016_v1.csv", sep=",", encoding = "latin-1")
# census_shp = gpd.read_file("/Users/robertdeng/Google Drive/Data Science/W209/Project/Tract_2010Census_DP1/Tract_2010Census_DP1.shp")

Let's first check the availability of state data and store the missing url requests in a list

In [3]:
full_state_test = ["%.2d" % i for i in range(1,57)]
bad_apples = []
def state_checker(full_state_test):
    for i in full_state_test:
        url = ("https://api.census.gov/data/2015/acs5?get=NAME,B03002_001E"+
               "&for=tract:*&in=state:" + i + "&key=14ba39dd26088efd8d54c4f01d90023f2d4bfc6d")
        response_code = requests.get(url).status_code
        if response_code != 200:
            bad_apples.append([i, response_code])
state_checker(full_state_test)
print("These states return no content. Bad Apples :(\n", bad_apples)

These states return no content. Bad Apples :(
 [['03', 204], ['07', 204], ['14', 204], ['43', 204], ['52', 204]]


3, 7, 14, 43, 52 are missing, but all the states should be in there if we go up to 56

https://www.census.gov/geo/reference/ansi_statetables.html

In [4]:
def pull_census(state, url_yes_no):
    url = ("https://api.census.gov/data/2015/acs5?get=NAME,B03002_001E,B03002_004E,B03002_006E," +
           "B03002_007E,B03002_008E,B03002_010E,B03002_012E," + #raceethicity
           "B05001_001E,B05001_006E," + #citizenship
           "B05002_013E" #foreign born
           "B05010_001E,B05010_002E," + #povertyline
           "B07009_001E,B07009_002E,B07009_003E,B07009_004E,B07009_006E," + #edu
           "B07013_001E,B07013_003E,"+ #renter
           "B01001_001E,B01001_003E,B01001_027E" + #under5 
           "&for=tract:*&in=state:" + state + "&key=14ba39dd26088efd8d54c4f01d90023f2d4bfc6d")
    if url_yes_no:
        print(url)        
    html = requests.get(url).json()
    return html

In [11]:
# pull_census("01", False)

In [6]:
#Make a master list range and remove the bad apples
master_list = ["%.2d" % i for i in range(1,57)]
master_list = [i for i in master_list if i not in [bad_apples[i][0] for i in range(len(bad_apples))]]

#Then stitch together all the data frames for the remaining dataset
for i in master_list:
    if i == "01":
        newstate = pull_census(i, False)
        master = pd.DataFrame(newstate, columns = newstate[0])[1:]
    elif i != "01":
        newstate = pull_census(i, False)
        master = master.append(pd.DataFrame(newstate, columns = newstate[0])[1:])

In [7]:
#Column Creation
master["GEOID"] = master['state'] + master['county'] + master['tract']
master["County Name"] = master["NAME"].str.split(",").str[1]
master["State Name"] = master["NAME"].str.split(",").str[2]

In [9]:
master.columns = ['Name', 'Total_all_race', 'Black', 'Asian', 'Native_Hawaiian_Pacific_Islander', 'Other', 'Two_or_More_Races', 'Hispanic_or_Latino', 
                  'Total_citizen', 'Not_a_us_Citizen',
                  'Total_born', 'Total_foreign_born',
                  'Total_income_population', "Total_under_poverty_line",
                  'Total_edu','Less_than_HS', 'HS_grad', 'College_grad', 'Graduate_or_professional',
                  'Total_housing', 'renters', 
                  'Total_pop_all_age','Total_male_under5','Total_female_under5',
                  'state', 'county', 'tract', 'GEOID', 'County Name', 'State Name']

In [10]:
master.head()

,Name,Total_all_race,Black,Asian,Native_Hawaiian_Pacific_Islander,Other,Two_or_More_Races,Hispanic_or_Latino,Total_citizen,Not_a_us_Citizen,...,renters,Total_pop_all_age,Total_male_under5,Total_female_under5,state,county,tract,GEOID,County Name,State Name
1,"Census Tract 201, Autauga County, Alabama",1948,150,12,0,0,0,17,1948,27,...,538,1948,30,46,01,001,020100,01001020100,Autauga County,Alabama
2,"Census Tract 202, Autauga County, Alabama",2156,1149,50,0,0,0,17,2156,37,...,754,2156,26,40,01,001,020200,01001020200,Autauga County,Alabama
3,"Census Tract 203, Autauga County, Alabama",2968,551,41,8,0,0,0,2968,15,...,864,2968,38,25,01,001,020300,01001020300,Autauga County,Alabama
4,"Census Tract 204, Autauga County, Alabama",4423,162,0,0,48,5,464,4423,33,...,836,4423,102,93,01,001,020400,01001020400,Autauga County,Alabama
5,"Census Tract 205, Autauga County, Alabama",10763,2674,412,0,0,49,80,10763,188,...,4611,10763,403,402,01,001,020500,01001020500,Autauga County,Alabama


In [37]:
pdb = pd.read_csv("pdb2016trv8_us.csv", sep = ",", encoding = "latin-1",dtype={'GIDTR':object})

In [38]:
pdb.head()

,GIDTR,State,State_name,County,County_name,Tract,Flag,Num_BGs_in_Tract,LAND_AREA,AIAN_LAND,...,pct_TEA_MailOutMailBack_CEN_2010,pct_TEA_Update_Leave_CEN_2010,pct_Census_Mail_Returns_CEN_2010,pct_Vacants_CEN_2010,pct_Deletes_CEN_2010,pct_Census_UAA_CEN_2010,pct_Mailback_Count_CEN_2010,pct_FRST_FRMS_CEN_2010,pct_RPLCMNT_FRMS_CEN_2010,pct_BILQ_Mailout_count_CEN_2010
0,01001020100,1,Alabama,1,Autauga County,20100,NaN,2.0,3.788,0.0,...,100.0,NaN,68.25,1.92,0.0,16.39,81.69,61.84,6.40,NaN
1,01001020200,1,Alabama,1,Autauga County,20200,NaN,2.0,1.290,0.0,...,100.0,NaN,68.82,2.28,0.0,13.07,84.65,60.79,8.03,NaN
2,01001020300,1,Alabama,1,Autauga County,20300,NaN,2.0,2.065,0.0,...,100.0,NaN,72.95,1.67,0.0,6.53,91.79,72.95,0.00,NaN
3,01001020400,1,Alabama,1,Autauga County,20400,NaN,4.0,2.464,0.0,...,100.0,NaN,77.64,1.46,0.0,5.51,93.03,77.64,0.00,NaN
4,01001020500,1,Alabama,1,Autauga County,20500,NaN,3.0,4.401,0.0,...,100.0,NaN,70.97,2.16,0.0,5.96,91.87,70.97,0.00,NaN


In [17]:
# pdb['Low_Response_Score']

In [18]:
# list(pdb.columns.values)

In [23]:
# pdb['pct_Mailback_Count_CEN_2010']

In [88]:
# pdb2=pdb[['GIDTR','pct_Mailback_Count_CEN_2010','Low_Response_Score']]  # df2 but only with columns x, a, and b

# pdb2

In [43]:
master_merged = master.merge(pdb2, left_on = "GEOID", right_on = "GIDTR", how = "left")

In [44]:
master_merged

,Name,Total_all_race,Black,Asian,Native_Hawaiian_Pacific_Islander,Other,Two_or_More_Races,Hispanic_or_Latino,Total_citizen,Not_a_us_Citizen,...,Total_female_under5,state,county,tract,GEOID,County Name,State Name,GIDTR,pct_Mailback_Count_CEN_2010,Low_Response_Score
0,"Census Tract 201, Autauga County, Alabama",1948,150,12,0,0,0,17,1948,27,...,46,01,001,020100,01001020100,Autauga County,Alabama,01001020100,81.69,17.9
1,"Census Tract 202, Autauga County, Alabama",2156,1149,50,0,0,0,17,2156,37,...,40,01,001,020200,01001020200,Autauga County,Alabama,01001020200,84.65,23.2
2,"Census Tract 203, Autauga County, Alabama",2968,551,41,8,0,0,0,2968,15,...,25,01,001,020300,01001020300,Autauga County,Alabama,01001020300,91.79,17.9
3,"Census Tract 204, Autauga County, Alabama",4423,162,0,0,48,5,464,4423,33,...,93,01,001,020400,01001020400,Autauga County,Alabama,01001020400,93.03,13.5
4,"Census Tract 205, Autauga County, Alabama",10763,2674,412,0,0,49,80,10763,188,...,402,01,001,020500,01001020500,Autauga County,Alabama,01001020500,91.87,19.0
5,"Census Tract 206, Autauga County, Alabama",3851,459,0,0,0,0,503,3851,88,...,123,01,001,020600,01001020600,Autauga County,Alabama,01001020600,82.91,20.2
6,"Census Tract 207, Autauga County, Alabama",2761,543,0,0,31,0,104,2761,11,...,213,01,001,020700,01001020700,Autauga County,Alabama,01001020700,81.61,22.8
7,"Census Tract 208.01, Autauga County, Alabama",3187,341,0,0,0,0,40,3187,0,...,38,01,001,020801,01001020801,Autauga County,Alabama,01001020801,84.32,18.5
8,"Census Tract 208.02, Autauga County, Alabama",10915,917,0,0,0,0,150,10915,24,...,319,01,001,020802,01001020802,Autauga County,Alabama,01001020802,84.49,18.2
9,"Census Tract 209, Autauga County, Alabama",5668,688,19,0,0,0,23,5668,23,...,191,01,001,020900,01001020900,Autauga County,Alabama,01001020900,77.27,20.0


In [48]:
fcc = pd.read_csv("tract_map_jun_2016.csv", sep=",", encoding = "latin-1", dtype={'tractcode': object})

In [89]:
# fcc.head()

In [51]:
master_merged_final = master_merged.merge(fcc, left_on = "GEOID", right_on = "tractcode", how = "left")

In [52]:
master_merged_final

,Name,Total_all_race,Black,Asian,Native_Hawaiian_Pacific_Islander,Other,Two_or_More_Races,Hispanic_or_Latino,Total_citizen,Not_a_us_Citizen,...,tract,GEOID,County Name,State Name,GIDTR,pct_Mailback_Count_CEN_2010,Low_Response_Score,tractcode,pcat_all,pcat_10x1
0,"Census Tract 201, Autauga County, Alabama",1948,150,12,0,0,0,17,1948,27,...,020100,01001020100,Autauga County,Alabama,01001020100,81.69,17.9,01001020100,4.0,4.0
1,"Census Tract 202, Autauga County, Alabama",2156,1149,50,0,0,0,17,2156,37,...,020200,01001020200,Autauga County,Alabama,01001020200,84.65,23.2,01001020200,4.0,3.0
2,"Census Tract 203, Autauga County, Alabama",2968,551,41,8,0,0,0,2968,15,...,020300,01001020300,Autauga County,Alabama,01001020300,91.79,17.9,01001020300,5.0,4.0
3,"Census Tract 204, Autauga County, Alabama",4423,162,0,0,48,5,464,4423,33,...,020400,01001020400,Autauga County,Alabama,01001020400,93.03,13.5,01001020400,5.0,4.0
4,"Census Tract 205, Autauga County, Alabama",10763,2674,412,0,0,49,80,10763,188,...,020500,01001020500,Autauga County,Alabama,01001020500,91.87,19.0,01001020500,5.0,4.0
5,"Census Tract 206, Autauga County, Alabama",3851,459,0,0,0,0,503,3851,88,...,020600,01001020600,Autauga County,Alabama,01001020600,82.91,20.2,01001020600,5.0,4.0
6,"Census Tract 207, Autauga County, Alabama",2761,543,0,0,31,0,104,2761,11,...,020700,01001020700,Autauga County,Alabama,01001020700,81.61,22.8,01001020700,4.0,4.0
7,"Census Tract 208.01, Autauga County, Alabama",3187,341,0,0,0,0,40,3187,0,...,020801,01001020801,Autauga County,Alabama,01001020801,84.32,18.5,01001020801,4.0,3.0
8,"Census Tract 208.02, Autauga County, Alabama",10915,917,0,0,0,0,150,10915,24,...,020802,01001020802,Autauga County,Alabama,01001020802,84.49,18.2,01001020802,5.0,4.0
9,"Census Tract 209, Autauga County, Alabama",5668,688,19,0,0,0,23,5668,23,...,020900,01001020900,Autauga County,Alabama,01001020900,77.27,20.0,01001020900,3.0,2.0


In [59]:
list(master_merged_final.columns.values)

['Name',
 'Total_all_race',
 'Black',
 'Asian',
 'Native_Hawaiian_Pacific_Islander',
 'Other',
 'Two_or_More_Races',
 'Hispanic_or_Latino',
 'Total_citizen',
 'Not_a_us_Citizen',
 'Total_income_population',
 'Total_under_poverty_line',
 'Total_edu',
 'Less_than_HS',
 'HS_grad',
 'College_grad',
 'Graduate_or_professional',
 'Total_housing',
 'renters',
 'Total_pop_all_age',
 'Total_male_under5',
 'Total_female_under5',
 'state',
 'county',
 'tract',
 'GEOID',
 'County Name',
 'State Name',
 'GIDTR',
 'pct_Mailback_Count_CEN_2010',
 'Low_Response_Score',
 'tractcode',
 'pcat_all',
 'pcat_10x1']

In [56]:
a=master_merged_final

In [67]:
master_merged_final[['Total_all_race',
 'Black',
 'Asian',
 'Native_Hawaiian_Pacific_Islander',
 'Other',
 'Two_or_More_Races',
 'Hispanic_or_Latino',
 'Total_citizen',
 'Not_a_us_Citizen',
 'Total_income_population',
 'Total_under_poverty_line',
 'Total_edu',
 'Less_than_HS',
 'HS_grad',
 'College_grad',
 'Graduate_or_professional',
 'Total_housing',
 'renters',
 'Total_pop_all_age',
 'Total_male_under5',
 'Total_female_under5',
 'pct_Mailback_Count_CEN_2010',
 'Low_Response_Score',
'pcat_all',
 'pcat_10x1']] = master_merged_final[['Total_all_race',
 'Black',
 'Asian',
 'Native_Hawaiian_Pacific_Islander',
 'Other',
 'Two_or_More_Races',
 'Hispanic_or_Latino',
 'Total_citizen',
 'Not_a_us_Citizen',
 'Total_income_population',
 'Total_under_poverty_line',
 'Total_edu',
 'Less_than_HS',
 'HS_grad',
 'College_grad',
 'Graduate_or_professional',
 'Total_housing',
 'renters',
 'Total_pop_all_age',
 'Total_male_under5',
 'Total_female_under5',
 'pct_Mailback_Count_CEN_2010',
 'Low_Response_Score',
'pcat_all',
 'pcat_10x1']].apply(pd.to_numeric)

In [68]:
a=master_merged_final

In [76]:
a['Total_minority']=a['Black']+a['Asian']+a['Native_Hawaiian_Pacific_Islander']+a['Other']+a['Two_or_More_Races']+a['Hispanic_or_Latino']

In [80]:
a['pct_minority']=a['Total_minority']/a['Total_all_race']

In [82]:
a['pct_not_citizen']=a['Not_a_us_Citizen']/ a['Total_citizen']

In [84]:
a['Total_under_5']=a['Total_male_under5']+a['Total_female_under5']

In [85]:
a['pct_under_5']=a['Total_under_5']/a['Total_pop_all_age']

In [86]:
a['pct_renters']=a['renters']/a['Total_housing']

In [81]:
a.pct_minority.quantile(0.8)

0.648477233213002

In [87]:
a

,Name,Total_all_race,Black,Asian,Native_Hawaiian_Pacific_Islander,Other,Two_or_More_Races,Hispanic_or_Latino,Total_citizen,Not_a_us_Citizen,...,Low_Response_Score,tractcode,pcat_all,pcat_10x1,Total_minority,pct_minority,pct_not_citizen,Total_under_5,pct_under_5,pct_renters
0,"Census Tract 201, Autauga County, Alabama",1948,150,12,0,0,0,17,1948,27,...,17.9,01001020100,4.0,4.0,179,0.091889,0.013860,76,0.039014,0.277606
1,"Census Tract 202, Autauga County, Alabama",2156,1149,50,0,0,0,17,2156,37,...,23.2,01001020200,4.0,3.0,1216,0.564007,0.017161,66,0.030612,0.381001
2,"Census Tract 203, Autauga County, Alabama",2968,551,41,8,0,0,0,2968,15,...,17.9,01001020300,5.0,4.0,600,0.202156,0.005054,63,0.021226,0.294479
3,"Census Tract 204, Autauga County, Alabama",4423,162,0,0,48,5,464,4423,33,...,13.5,01001020400,5.0,4.0,679,0.153516,0.007461,195,0.044088,0.190955
4,"Census Tract 205, Autauga County, Alabama",10763,2674,412,0,0,49,80,10763,188,...,19.0,01001020500,5.0,4.0,3215,0.298709,0.017467,805,0.074793,0.440738
5,"Census Tract 206, Autauga County, Alabama",3851,459,0,0,0,0,503,3851,88,...,20.2,01001020600,5.0,4.0,962,0.249805,0.022851,257,0.066736,0.288400
6,"Census Tract 207, Autauga County, Alabama",2761,543,0,0,31,0,104,2761,11,...,22.8,01001020700,4.0,4.0,678,0.245563,0.003984,284,0.102861,0.342174
7,"Census Tract 208.01, Autauga County, Alabama",3187,341,0,0,0,0,40,3187,0,...,18.5,01001020801,4.0,3.0,381,0.119548,0.000000,91,0.028553,0.180761
8,"Census Tract 208.02, Autauga County, Alabama",10915,917,0,0,0,0,150,10915,24,...,18.2,01001020802,5.0,4.0,1067,0.097755,0.002199,748,0.068530,0.192890
9,"Census Tract 209, Autauga County, Alabama",5668,688,19,0,0,0,23,5668,23,...,20.0,01001020900,3.0,2.0,730,0.128793,0.004058,451,0.079570,0.178629


# Shapefile

In [ ]:
#Get Polygon coordinates
def getPolyCoords(row, geom, coord_type):
    """Returns the coordinates ('x' or 'y') of edges of a Polygon exterior"""
    if row[geom].type == 'Polygon':
    # Parse the exterior of the coordinate
        exterior = row[geom].exterior

        if coord_type == 'x':
        # Get the x coordinates of the exterior
            return list( exterior.coords.xy[0] )
        elif coord_type == 'y':
        # Get the y coordinates of the exterior
            return list( exterior.coords.xy[1] )       

#Create x, y coordinates for polygons
census_shp["x"] = census_shp.apply(getPolyCoords, geom="geometry", coord_type="x", axis=1)
census_shp["y"] = census_shp.apply(getPolyCoords, geom="geometry", coord_type="y", axis=1)

#x, y ranges
tract_bounds=census_shp.total_bounds
xlim = (tract_bounds[0],tract_bounds[2])
ylim = (tract_bounds[1],tract_bounds[3])

bounds = x_range,y_range = (xlim,ylim)

In [ ]:
#NaN Checks
#nan_index = merged.index[np.isnan(merged.state.astype(float))].tolist()
#len(merged.index[np.isnan(merged.state.astype(float))].tolist())

In [ ]:
merged = census_shp.merge(master, left_on = "GEOID10", right_on = "GEOID", how = "left", sort = True)
#merged.head()

In [ ]:
#import pysal as ps
#pop = merged.Total_Population.astype(float)
#pop = pop[~np.isnan(pop)]

In [ ]:
#popq = ps.Quantiles(pop,k=9)
#popq, popq.adcm

In [ ]:
#popeq = ps.Equal_Interval(pop,k=9)
#popeq, popeq.adcm

In [ ]:
#popfj = ps.Fisher_Jenks(pop,k=9)
#popfj, popfj.adcm

In [ ]:
temp_stored_merged = merged
merged = merged[merged["state"] == "06"]

#http://geographyplanning.buffalostate.edu/Mix/Python/Visualization-PoorChildrenNY.html

merged_ys = merged['y'].tolist()
merged_xs = merged['x'].tolist()
merged_name = merged['NAMELSAD10'].tolist()
merged_pop = merged['Total_Population'].tolist()

color_mapper = LogColorMapper(palette=palette)

source = ColumnDataSource(
        data=dict(
            x=merged_xs,
            y=merged_ys,
            name=merged_name,
            population = merged_pop
        )
    )

TOOLS = "pan,wheel_zoom,box_zoom,reset,hover,save"

f = figure(
    plot_height=700, plot_width=800,
    title="Census 5 Year Test",
    tools=TOOLS,
    x_axis_location=None,
    y_axis_location=None,
    x_range=x_range,
    y_range=y_range)

f.title.text_font_style = "italic"
f.title.text_font_size = '14pt'
f.background_fill_color = 'gray'
f.grid.grid_line_color = None
f.add_tile(STAMEN_TERRAIN)

f.patches("x", "y",source=source, fill_color = {'field': 'population', 'transform': color_mapper},
          fill_alpha=0.7,line_color = 'white',line_width=0.5) 

hover = f.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [
    ("Census Tract", "@name"),
    ("Population", "@population"),
]
show(f)